In [1]:
import pandas as pd
import numpy as np
import time
# pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

## TO DO: LOOK FOR MAX VALUES BETWEEN PASS THROW AND PASS CATCH INSTEAD OF THROUGH ALL

# WEEKS = list(range(1, 18))
WEEKS = [16]

og_df = pd.DataFrame()

#ingest
for week in WEEKS:
    og_df = og_df.append(pd.read_csv(f"../data/week{week}_norm.csv", low_memory=False))
# data = pd.read_csv('./data/weektoy.csv', low_memory=False)
# track_df.reset_index(drop=True, inplace=True)
og_df.head()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,team,team_pos,teamAbbr,route,time,los,x,y,dis,o,s,s_dir,s_dir_rad,v_x,v_y,v_theta,v_mag,a_old,a_x,a_y,a_theta,a_mag
0,2018122200,88,1,None,264.0,Josh Johnson,8.0,QB,away,OFF,WAS,NaN,2018-12-22T21:31:56.500Z,40,38.73,23.78,0.00,78.10,0.00,21.27,0.37,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
1,2018122200,88,1,None,2354.0,Wesley Woodyard,59.0,ILB,home,DEF,TEN,NaN,2018-12-22T21:31:56.500Z,40,44.33,24.08,0.07,274.14,0.66,196.09,3.42,-0.18,-0.63,1.29,0.66,0.52,0.0,0.0,0.0,0.0
2,2018122200,88,1,None,496828.0,Derrick Morgan,91.0,OLB,home,DEF,TEN,NaN,2018-12-22T21:31:56.500Z,40,41.05,18.30,0.01,299.04,0.01,20.48,0.36,0.00,0.01,1.21,0.01,0.01,0.0,0.0,0.0,0.0
3,2018122200,88,1,None,2507164.0,Adrian Peterson,26.0,RB,away,OFF,WAS,OUT,2018-12-22T21:31:56.500Z,40,32.06,23.91,0.00,120.07,0.00,166.34,2.90,0.00,-0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,2018122200,88,1,None,2532841.0,Michael Floyd,17.0,WR,away,OFF,WAS,GO,2018-12-22T21:31:56.500Z,40,36.87,19.78,0.34,143.48,3.31,175.90,3.07,0.24,-3.30,-1.50,3.31,1.40,0.0,0.0,0.0,0.0


In [2]:
df = og_df.copy().loc[og_df.nflId != 0]
len(df)

1066001

In [3]:
df = df[['gameId', 'nflId', 'displayName', 'position', 'playId', 'frameId', 'v_mag', 'a_mag']]

In [4]:
mapping = {'DB': 'DB', 'CB': 'DB', 'S': 'S', 'SS': 'S', 'FS': 'S', 'WR': 'WR', 'MLB': 'LB', 'OLB': 'LB', 'ILB': 'LB', 'LB': 'LB', 'DL': 'DL', 'DT':'DL', 'DE': 'DL', 'NT': 'DL', 'QB': 'QB', 'RB': 'RB', 'HB': 'RB', 'TE': 'TE', 'P': 'ST', 'K': 'ST', 'LS':'ST', 'FB': 'FB'}

df['position_category'] = df.position.map(mapping)

In [5]:
set(df.position.unique()) - set(mapping.keys()) 

set()

In [12]:
df2 = df.copy()
start = time.time()
def v_roll(x):
    return x[['v_mag']].rolling(20).apply(np.mean)

res = df2.groupby(['gameId', 'playId', 'nflId']).apply(v_roll)
# df['roll_a_10'] = df.groupby(['gameId', 'playId', 'nflId'])['a_mag'].transform(lambda x: x.rolling(10).parallel_apply(np.mean))
print(time.time() - start)
res.tail(110)
# df.head()

129.6866216659546


v_mag
gameId     playId nflId                    
2018122400 4129   2560888.0 1144034  2.9850
                  2561002.0 1142415     NaN
                            1142430     NaN
                            1142445     NaN
                            1142460     NaN
...                                     ...
                            1143975  3.0310
                            1143990  3.0390
                            1144005  3.0450
                            1144020  3.0545
                            1144035  3.0620

[110 rows x 1 columns]

In [7]:
df2 = df.copy()
start = time.time()
res2 = df2.groupby(['gameId', 'playId', 'nflId'])['v_mag'].rolling(20).parallel_apply(np.mean)
print(time.time() - start)
res2.tail(110)

In [99]:
res.tail(110)

gameId      playId  nflId             
2018122400  4129    2560888.0  1144034    2.9850
                    2561002.0  1142415       NaN
                               1142430       NaN
                               1142445       NaN
                               1142460       NaN
                                           ...  
                               1143975    3.0310
                               1143990    3.0390
                               1144005    3.0450
                               1144020    3.0545
                               1144035    3.0620
Name: v_mag, Length: 110, dtype: float64

In [85]:
start = time.time()
df['roll_v_20'] = df.groupby(['gameId', 'playId', 'nflId'])['v_mag'].transform(lambda x: x.rolling(20).apply(np.mean))
df['roll_a_10'] = df.groupby(['gameId', 'playId', 'nflId'])['a_mag'].transform(lambda x: x.rolling(10).apply(np.mean))
print(time.time() - start)
df.tail()

262.05738735198975


,gameId,nflId,displayName,position,playId,frameId,v_mag,a_mag,position_category,roll_v_20,roll_a_10
0,2018122200,264.0,Josh Johnson,QB,88,1,0.00,0.0,QB,NaN,NaN
1,2018122200,2354.0,Wesley Woodyard,ILB,88,1,0.66,0.0,LB,NaN,NaN
2,2018122200,496828.0,Derrick Morgan,OLB,88,1,0.01,0.0,LB,NaN,NaN
3,2018122200,2507164.0,Adrian Peterson,RB,88,1,0.00,0.0,RB,NaN,NaN
4,2018122200,2532841.0,Michael Floyd,WR,88,1,3.31,0.0,WR,NaN,NaN


In [97]:
df.loc[df.nflId==2561002.0].tail(110)

,gameId,nflId,displayName,position,playId,frameId,v_mag,a_mag,position_category,roll_v_20,roll_a_10
1142400,2018122400,2561002.0,DaeSean Hamilton,WR,4073,55,2.96,4.69,WR,3.4775,3.005
1142415,2018122400,2561002.0,DaeSean Hamilton,WR,4129,1,0.00,0.00,WR,NaN,NaN
1142430,2018122400,2561002.0,DaeSean Hamilton,WR,4129,2,0.00,0.00,WR,NaN,NaN
1142445,2018122400,2561002.0,DaeSean Hamilton,WR,4129,3,0.00,0.00,WR,NaN,NaN
1142460,2018122400,2561002.0,DaeSean Hamilton,WR,4129,4,0.00,0.00,WR,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1143975,2018122400,2561002.0,DaeSean Hamilton,WR,4129,105,3.15,0.36,WR,3.0310,0.578
1143990,2018122400,2561002.0,DaeSean Hamilton,WR,4129,106,3.14,0.31,WR,3.0390,0.556
1144005,2018122400,2561002.0,DaeSean Hamilton,WR,4129,107,3.15,0.55,WR,3.0450,0.547
1144020,2018122400,2561002.0,DaeSean Hamilton,WR,4129,108,3.14,0.12,WR,3.0545,0.495


In [8]:
len(df)

1066001

In [53]:
play_maxes = df.groupby(['gameId', 'playId', 'nflId', 'displayName', 'position', 'position_category'], as_index=False)['roll_v_20'].max()[['nflId', 'displayName','position', 'position_category', 'roll_v_20']]
play_maxes.tail()

,nflId,displayName,position,position_category,roll_v_20
16522,2558985.0,Tim Patrick,WR,WR,8.4735
16523,2559054.0,Nicholas Morrow,ILB,LB,7.0255
16524,2560733.0,Royce Freeman,RB,RB,7.8760
16525,2560888.0,Courtland Sutton,WR,WR,9.1715
16526,2561002.0,DaeSean Hamilton,WR,WR,7.8020


In [54]:
player_maxes = play_maxes.groupby(['nflId', 'displayName', 'position', 'position_category'],  as_index=False).max().dropna().set_index(['nflId'])

In [55]:
def rename(newname):
    def decorator(f):
        f.__name__ = newname
        return f
    return decorator

def q_at(y):
    @rename(f'Q{y:0.2f}')
    def q(x):
        return x.quantile(y)
    return q

stats_cat = player_maxes[['displayName', 'position', 'position_category', 'roll_v_20']].groupby('position_category').agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)
stats_cat['player_count'] = player_maxes.groupby('position_category')['displayName'].count()

stats_pos = player_maxes[['displayName', 'position', 'position_category', 'roll_v_20']].groupby('position').agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)
stats_pos['player_count'] = player_maxes.groupby('position')['displayName'].count()

In [58]:
stats_cat.sort_values(by=('roll_v_20', 'Q0.60'), ascending=False).style.background_gradient(cmap='viridis').format("{:4.2f}")

In [13]:
stats_pos.sort_values(by=('roll_a_10', 'Q0.60'), ascending=False).style.background_gradient(cmap='viridis').format("{:4.2f}")

In [14]:
maxes.loc[maxes.position=='QB'].sort_values('roll_v_20', ascending=False).reset_index(drop=True).head(20).style.background_gradient(cmap='viridis')

,displayName,position,position_category,roll_v_20,roll_a_10
0,Jameis Winston,QB,QB,9.184500,6.322000
1,Patrick Mahomes,QB,QB,8.668500,6.232000
2,Lamar Jackson,QB,QB,8.310000,7.313000
3,Taysom Hill,QB,QB,8.099500,5.952000
4,Taylor Heinicke,QB,QB,7.924500,6.519000
5,Baker Mayfield,QB,QB,7.810500,5.669000
6,Case Keenum,QB,QB,7.629500,6.142000
7,Blaine Gabbert,QB,QB,7.492000,5.953000
8,Aaron Rodgers,QB,QB,7.484000,6.770000
9,Mitchell Trubisky,QB,QB,7.392000,7.779000


In [15]:
# df = df[['gameId', 'nflId', 'displayName', 'position', 'playId', 'frameId', 'v_mag', 'a_mag']]

df.groupby('nflId')['playId'].nunique().describe()

count    731.000000
mean      22.608755
std       15.011690
min        1.000000
25%        9.000000
50%       23.000000
75%       33.000000
max       60.000000
Name: playId, dtype: float64

In [16]:
stats_cat.player_count.sum()

731

In [70]:
bin_df = df.copy()
bin_df['v_bin'] = pd.cut(bin_df.v_mag, bins=[-1, 6, 8, 12])
bin_df.tail()

,gameId,nflId,displayName,position,playId,frameId,v_mag,a_mag,position_category,roll_v_20,roll_a_10,v_bin
1144031,2018122400,2558985.0,Tim Patrick,WR,4129,109,6.00,3.11,WR,7.2370,2.392,"(-1, 6]"
1144032,2018122400,2559054.0,Nicholas Morrow,ILB,4129,109,6.38,0.76,LB,6.1945,0.482,"(6, 8]"
1144033,2018122400,2560733.0,Royce Freeman,RB,4129,109,7.21,3.15,RB,6.8225,1.732,"(6, 8]"
1144034,2018122400,2560888.0,Courtland Sutton,WR,4129,109,2.70,0.89,WR,2.9850,0.578,"(-1, 6]"
1144035,2018122400,2561002.0,DaeSean Hamilton,WR,4129,109,3.12,0.25,WR,3.0620,0.450,"(-1, 6]"


In [71]:
player_maxes = bin_df[['displayName', 'position', 'position_category', 'v_bin', 'roll_a_10']].groupby(['displayName', 'v_bin'], as_index=False).max()
player_maxes.loc[player_maxes.displayName=='Tyreek Hill']

,displayName,v_bin,position,position_category,roll_a_10
2085,Tyreek Hill,"(-1, 6]",WR,WR,7.251
2086,Tyreek Hill,"(6, 8]",WR,WR,6.792
2087,Tyreek Hill,"(8, 12]",WR,WR,5.922


In [72]:
stats_cat = player_maxes.groupby(['position_category', 'v_bin'], as_index=False).agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)

stats_pos = player_maxes.groupby(['position', 'v_bin'], as_index=False).agg([q_at(.5), q_at(.6), q_at(.7), q_at(.8), q_at(.9), q_at(.95), q_at(.99)]).round(2)

In [73]:
stats_cat.loc[['WR', 'DB', 'S', 'RB', 'TE', 'LB', 'FB', 'QB', 'DL'], :].dropna().style.background_gradient(cmap='viridis').format("{:4.2f}")

In [74]:
stats_cat.sort_values(by=('roll_a_10', 'Q0.60'), ascending=False).style.background_gradient(cmap='viridis').format("{:4.2f}")

In [ ]:
# BELOW THIS IS EXPERIMENTAL STUFF IM KEEPIGN FOR REFERENCE

In [12]:
motion = track_df[['gameId', 'playId', 'position', 's', 'a']]
maxes = motion.groupby(['gameId', 'playId', 'position'], as_index=False).max()
maxes.head()


,gameId,playId,position,s,a
0,2018090600,75,CB,5.35,6.34
1,2018090600,75,FB,7.37,5.88
2,2018090600,75,FS,6.59,5.08
3,2018090600,75,LB,6.74,4.09
4,2018090600,75,MLB,5.64,4.49


In [13]:
stats = maxes.drop(['gameId', 'playId'], axis=1).groupby('position').describe(percentiles=[0.5, 0.8, 0.9, 0.95, 0.99])
stats

s                                                         \
            count      mean       std   min    50%    80%    90%     95%   
position                                                                   
CB        19197.0  7.007900  1.398815  0.04  6.990  8.240  8.830  9.2800   
DB         5136.0  5.974761  1.555471  0.36  5.940  7.300  8.040  8.6125   
DE          139.0  5.183094  1.272243  1.98  5.290  6.266  6.784  7.4420   
DL           37.0  5.023514  1.202024  2.25  5.120  6.012  6.518  6.9860   
DT           23.0  4.920435  1.611012  1.63  4.850  6.732  6.846  6.9870   
FB          632.0  5.901788  1.555494  1.23  6.095  7.040  7.519  8.0100   
FS        13909.0  6.115643  1.462061  0.23  6.140  7.340  7.980  8.5200   
HB          632.0  5.648940  1.559184  0.15  5.820  6.870  7.578  8.1045   
ILB        9440.0  5.904818  1.377072  0.11  5.920  7.100  7.670  8.1205   
K             5.0  4.534000  2.220277  0.96  4.770  5.894  6.402  6.6560   
LB         7153.0  5.768125  1.401809  0.17  5.740  6.980  7.590  8.1200   
LS           16.0  4.591875  2.049469  1.25  4.635  6.060  6.350  7.2375   
MLB        6227.0  5.657437  1.408919  0.00  5.620  6.840  7.484  8.0100   
NT            5.0  4.526000  1.185171  3.39  4.040  5.122  5.786  6.1180   
OLB       15863.0  5.704912  1.304016  0.11  5.660  6.770  7.420  7.9200   
P            16.0  4.441875  1.866409  1.39  4.285  6.360  6.665  6.9625   
QB        19199.0  3.437742  1.513756  0.05  3.050  4.620  5.590  6.4900   
RB        18267.0  5.547568  1.666610  0.00  5.650  6.940  7.560  8.1300   
S          1680.0  5.824839  1.526895  0.01  5.850  7.112  7.841  8.4005   
SS        12762.0  5.915921  1.521527  0.00  5.940  7.200  7.830  8.3600   
TE        18693.0  6.101853  1.490295  0.00  6.330  7.320  7.820  8.1940   
WR        19147.0  7.759936  1.188221  0.00  7.830  8.740  9.160  9.4900   

                                a                                          \
              99%    max    count      mean       std   min    50%    80%   
position                                                                    
CB        10.0000  11.79  19197.0  5.674798  1.218220  0.04  5.640  6.600   
DB         9.4795  10.69   5136.0  4.760378  1.245351  0.73  4.690  5.800   
DE         8.1884   8.97    139.0  4.213165  1.181914  1.46  4.220  5.218   
DL         7.3908   7.56     37.0  5.016757  1.198754  2.24  4.980  5.630   
DT         7.5460   7.70     23.0  4.034348  1.116540  1.32  3.850  4.880   
FB         8.8145  23.17    632.0  5.078275  1.969096  1.42  4.875  5.976   
FS         9.3600  24.66  13909.0  4.825936  1.243047  0.40  4.800  5.770   
HB         8.8000  10.15    632.0  4.962120  1.334472  0.29  4.830  5.890   
ILB        8.8561  10.67   9440.0  4.984284  1.249841  0.34  4.920  5.940   
K          6.8592   6.91      5.0  2.836000  0.870419  1.41  2.900  3.484   
LB         8.9048  10.75   7153.0  4.919796  1.226860  0.29  4.870  5.840   
LS         8.8155   9.21     16.0  2.886250  0.746993  1.64  2.845  3.540   
MLB        8.9074  10.38   6227.0  4.739489  1.216272  0.00  4.670  5.690   
NT         6.3836   6.45      5.0  3.426000  0.681601  2.49  3.480  3.840   
OLB        8.8338  11.34  15863.0  5.192158  1.374139  0.32  5.070  6.150   
P          7.1125   7.15     16.0  3.435000  1.736295  1.39  2.995  4.240   
QB         7.8800   9.77  19199.0  3.720094  1.370266  0.05  3.540  4.680   
RB         9.0900  42.74  18267.0  4.797587  1.505950  0.00  4.660  5.770   
S          9.2605  10.68   1680.0  4.588518  1.222840  0.01  4.560  5.600   
SS         9.2500  11.28  12762.0  4.738746  1.240811  0.00  4.720  5.748   
TE         8.8200  32.71  18693.0  5.151765  1.472649  0.00  5.020  6.080   
WR        10.0254  12.54  19147.0  6.449891  1.575455  0.00  6.170  7.270   

                                         
            90%     95%      99%    max  
position                                 
CB        7.120  7.5900   8.8300  30.73  
DB        6.380  6.88

In [14]:
stats.drop([('s', 'std'), ('s', 'min'), ('s', 'max'), ('s', 'mean'), ('a', 'std'), ('a', 'min'), ('a', 'count'), ('a', 'max'), ('a', 'mean')], axis=1).sort_values(by=('a', '95%'), ascending=False).round(2)

s                                    a                    \
            count   50%   80%   90%   95%    99%   50%   80%   90%   95%   
position                                                                   
WR        19147.0  7.83  8.74  9.16  9.49  10.03  6.17  7.27  8.25  9.68   
TE        18693.0  6.33  7.32  7.82  8.19   8.82  5.02  6.08  6.80  7.59   
CB        19197.0  6.99  8.24  8.83  9.28  10.00  5.64  6.60  7.12  7.59   
OLB       15863.0  5.66  6.77  7.42  7.92   8.83  5.07  6.15  6.83  7.57   
FB          632.0  6.10  7.04  7.52  8.01   8.81  4.88  5.98  6.72  7.32   
RB        18267.0  5.65  6.94  7.56  8.13   9.09  4.66  5.77  6.44  7.16   
DL           37.0  5.12  6.01  6.52  6.99   7.39  4.98  5.63  6.18  7.09   
HB          632.0  5.82  6.87  7.58  8.10   8.80  4.83  5.89  6.60  7.05   
ILB        9440.0  5.92  7.10  7.67  8.12   8.86  4.92  5.94  6.49  7.04   
LB         7153.0  5.74  6.98  7.59  8.12   8.90  4.87  5.84  6.37  6.92   
DB         5136.0  5.94  7.30  8.04  8.61   9.48  4.69  5.80  6.38  6.88   
MLB        6227.0  5.62  6.84  7.48  8.01   8.91  4.67  5.69  6.24  6.76   
FS        13909.0  6.14  7.34  7.98  8.52   9.36  4.80  5.77  6.30  6.75   
SS        12762.0  5.94  7.20  7.83  8.36   9.25  4.72  5.75  6.27  6.70   
S          1680.0  5.85  7.11  7.84  8.40   9.26  4.56  5.60  6.13  6.60   
QB        19199.0  3.05  4.62  5.59  6.49   7.88  3.54  4.68  5.49  6.23   
DE          139.0  5.29  6.27  6.78  7.44   8.19  4.22  5.22  5.54  6.23   
P            16.0  4.28  6.36  6.66  6.96   7.11  3.00  4.24  4.90  5.92   
DT           23.0  4.85  6.73  6.85  6.99   7.55  3.85  4.88  5.65  5.83   
NT            5.0  4.04  5.12  5.79  6.12   6.38  3.48  3.84  4.08  4.20   
LS           16.0  4.64  6.06  6.35  7.24   8.82  2.84  3.54  3.58  3.78   
K             5.0  4.77  5.89  6.40  6.66   6.86  2.90  3.48  3.55  3.59   

                 
            99%  
position         
WR        12.04  
TE         9.82  
CB         8.83  
OLB        9.43  
FB         9.77  
RB         9.33  
DL         8.38  
HB         9.11  
ILB        8.46  
LB         8.42  
DB         7.79  
MLB        8.16  
FS         7.74  
SS         7.75  
S          7.55  
QB         8.02  
DE         7.59  
P          8.02  
DT         6.38  
NT         4.30  
LS         4.20  
K          3.61

In [ ]:
motion = track_df[['gameId', 'playId', 'nflId', 'displayName','position', 's', 'a']]
maxes = motion2.groupby(['nflId', 'gameId', 'playId', 'displayName', 'position'], as_index=False).agg(
    frames = ('s', 'count'),
    max_s = ('s', 'max'),
    max_10th_s = ('s', lambda x: x.nlargest(10).iloc[-1]),
    max_15th_s = ('s', lambda x: x.nlargest(15).iloc[-1]),
    max_20th_s = ('s', lambda x: x.nlargest(20).iloc[-1]),
    max_a = ('a', 'max'),
    max_10th_a = ('a', lambda x: x.nlargest(10).iloc[-1]),
    max_15th_a = ('a', lambda x: x.nlargest(15).iloc[-1]),
    max_20th_a = ('a', lambda x: x.nlargest(20).iloc[-1]),
)

In [24]:
weeks = [15, 16]
toy = pd.DataFrame()
for week in weeks:
    toy = toy.append(pd.read_csv(f"../data/week{week}_norm.csv", low_memory=False))

In [56]:
df = toy.loc[toy.team != 'football']

df['roll_s_30'] = df.groupby(['gameId', 'playId', 'nflId'])['s'].transform(lambda x: x.rolling(30).mean())
df['roll_s_20'] = df.groupby(['gameId', 'playId', 'nflId'])['s'].transform(lambda x: x.rolling(20).mean())
df['roll_s_10'] = df.groupby(['gameId', 'playId', 'nflId'])['s'].transform(lambda x: x.rolling(10).mean())
df['roll_a_5'] = df.groupby(['gameId', 'playId', 'nflId'])['a'].transform(lambda x: x.rolling(5).mean())
df['roll_a_10'] = df.groupby(['gameId', 'playId', 'nflId'])['a'].transform(lambda x: x.rolling(5).mean())

In [61]:
motion = df[['gameId', 'playId', 'nflId', 'displayName','position', 's', 'roll_s_30', 'roll_s_20', 'roll_s_10','a', 'roll_a_5']]
maxes = motion.groupby(['nflId', 'displayName', 'position']).max()

# .agg(
#     frames = ('s', 'count'),
#     max_s = ('s', 'max'),
#     max_10th_s = ('s', lambda x: x.nlargest(10).iloc[-1]),
#     max_20th_s = ('s', lambda x: x.nlargest(20).iloc[-1]),
#     max_50th_s = ('s', lambda x: x.nlargest(50).iloc[-1]),
#     max_100th_s = ('s', lambda x: x.nlargest(100).iloc[-1]),
#     max_a = ('a', 'max'),
#     max_10th_a = ('a', lambda x: x.nlargest(10).iloc[-1]),
#     max_20th_a = ('a', lambda x: x.nlargest(20).iloc[-1]),
#     max_50th_a = ('a', lambda x: x.nlargest(50).iloc[-1]),
#     max_100th_a = ('a', lambda x: x.nlargest(100).iloc[-1]),
# )
# maxes.sort_values('max_s', ascending=False).head(20)

In [70]:
maxes['diff'] = maxes['roll_s_10']-maxes['roll_s_30']
maxes.sort_values('roll_s_20', ascending=False).head(100)
# maxes['diff'].describe()

,,,gameId,playId,s,roll_s_30,roll_s_20,roll_s_10,a,roll_a_5,diff
nflId,displayName,position,,,,,,,,,
2560725.0,Minkah Fitzpatrick,S,2018122302,3610,10.68,10.364000,10.4880,10.602,8.93,8.234,-0.238000
2552608.0,Stefon Diggs,WR,2018122307,3747,10.51,10.192000,10.3655,10.443,6.73,6.400,-0.251000
2543474.0,Darqueze Dennard,CB,2018122305,4235,10.35,10.214667,10.2425,10.248,7.18,6.736,-0.033333
2534436.0,Deonte Thompson,WR,2018122308,3130,10.53,10.132333,10.2295,10.338,8.79,7.242,-0.205667
2507166.0,Ted Ginn,WR,2018122313,4285,10.54,9.964333,10.2210,10.393,11.71,9.922,-0.428667
2558010.0,John Ross,WR,2018122305,4430,10.38,9.975667,10.1905,10.308,12.59,11.194,-0.332333
2543501.0,Aaron Colvin,CB,2018122310,4645,10.33,10.052333,10.1680,10.257,7.80,7.442,-0.204667
2543468.0,Mike Evans,WR,2018122306,4098,10.28,10.059667,10.1195,10.194,11.78,10.524,-0.134333
2559053.0,Torry McTyer,CB,2018122302,3610,10.35,9.712667,10.1005,10.228,6.67,6.366,-0.515333


In [3]:
pd.set_option('display.max_colwidth', 200)
# play_events = track_df[['gameId', 'playId', 'frameId', 'event']].groupby(['gameId', 'playId', 'frameId'], as_index=False).first().sort_values(by=['gameId', 'playId', 'frameId']).groupby(['gameId', 'playId']).agg({'frameId': 'nunique', 'event': lambda x: list(filter(lambda y: y!='None', x))})
play_events = track_df[['gameId', 'playId', 'frameId', 'event']].groupby(['gameId', 'playId', 'frameId'], as_index=False).first().sort_values(by=['gameId', 'playId', 'frameId']).groupby(['gameId', 'playId']).agg(frames = ('frameId', 'nunique'), events = ('event', lambda x: list(filter(lambda y: y!='None', x))))
play_events.head(20)

frames                                             events
gameId     playId                                                           
2018122200 88          80  [ball_snap, play_action, pass_forward, pass_ou...
           110         78  [ball_snap, pass_forward, pass_arrived, pass_o...
           163         43  [ball_snap, pass_forward, pass_outcome_incompl...
           218         83  [ball_snap, pass_forward, pass_arrived, pass_o...
           289         62  [ball_snap, play_action, pass_forward, pass_ar...
           358         77  [ball_snap, pass_forward, pass_arrived, pass_o...
           383         47  [ball_snap, pass_forward, pass_arrived, pass_o...
           478         66  [man_in_motion, ball_snap, pass_shovel, pass_a...
           787         95  [ball_snap, play_action, pass_forward, pass_ar...
           834         65  [ball_snap, pass_forward, pass_arrived, pass_o...
           904         52  [shift, ball_snap, pass_forward, pass_tipped, ...
           995         84  [ball_snap, play_action, pass_forward, pass_ar...
           1043        71                     [line_set, ball_snap, qb_sack]
           1085        99  [ball_snap, pass_forward, pass_arrived, pass_o...
           1176        63  [ball_snap, pass_forward, pass_arrived, pass_o...
           1322        74  [man_in_motion, ball_snap, play_action, pass_f...
           1353        67  [ball_snap, pass_forward, pass_arrived, pass_o...
           1400        64  [man_in_motion, ball_snap, play_action, pass_f...
           1479        51                  [ball_snap, play_action, qb_sack]
           1499        85  [shift, ball_snap, pass_forward, pass_arrived,...

In [4]:
ends = track_df.loc[(track_df.event=='pass_forward') | (track_df.event=='qb_sack') | (track_df.event=='pass_shovel')].groupby(['gameId', 'playId', 'frameId'], as_index=False).first()[['gameId', 'playId', 'frameId']].rename({'frameId': 'prePassEndFrame'}, axis=1).dropna(axis=0).set_index(['gameId', 'playId'])

track_df = track_df.join(ends, on=['gameId','playId'])

track_df.groupby(['gameId', 'playId']).first()[['frameId', 'event', 'displayName', 'prePassEndFrame']]

frameId      event       displayName  prePassEndFrame
gameId     playId                                                       
2018122200 88            1       None      Josh Johnson             53.0
           110           1       None      Josh Johnson             39.0
           163           1       None      Josh Johnson             27.0
           218           1       None      Josh Johnson             71.0
           289           1       None     Ryan Kerrigan             33.0
...                    ...        ...               ...              ...
2018122400 3900          1       None      Jordy Nelson             30.0
           3978          1       None  Marcus Gilchrist             26.0
           4003          1  ball_snap  Marcus Gilchrist             29.0
           4073          1       None  Marcus Gilchrist             36.0
           4129          1       None  Marcus Gilchrist             38.0

[1210 rows x 4 columns]

In [5]:
track_df = track_df.loc[(track_df.frameId >= 10) & (track_df.frameId <= track_df.prePassEndFrame)]
# track_df = track_df.loc[track_df.team_pos!='FTBL']

In [6]:
plays = list(track_df.groupby(['gameId', 'playId'], as_index=False).first()[['gameId', 'playId']].to_records(index=False))

In [7]:
# Select the game and play that you wish to see in week 1
import random

game_id = 2018122314
play_id = 4239

# game_id, play_id = random.choice(plays)

unique_game_ids = track_df.gameId.unique()
unique_play_ids = track_df[track_df.gameId == game_id].playId.unique()

play_df = track_df[track_df.playId == play_id].sort_values(by = 'time')
play_df.head()

,gameId,playId,frameId,event,nflId,displayName,jerseyNumber,position,team,team_pos,teamAbbr,route,time,los,x,y,s,dis,a,o,dir,dir_rad,v_x,v_y,v_theta,prePassEndFrame
1073926,2018122314,4239,10,None,497224.0,Ed Dickson,84.0,TE,home,OFF,SEA,GO,2018-12-24T04:26:43.400Z,40,38.37,19.16,0.00,0.00,0.00,80.57,268.01,4.677657,-0.000000,-0.000000,0.000000,39.0
1073939,2018122314,4239,10,None,NaN,Football,NaN,NaN,football,FTBL,FTBL,NaN,2018-12-24T04:26:43.400Z,40,39.52,23.70,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.000000,39.0
1073938,2018122314,4239,10,None,2561417.0,Charvarius Ward,35.0,DB,away,DEF,KC,NaN,2018-12-24T04:26:43.400Z,40,39.89,10.68,0.14,0.01,0.43,269.87,191.17,3.336546,-0.027121,-0.137348,1.375843,39.0
1073937,2018122314,4239,10,None,2561103.0,Tremon Smith,39.0,DB,away,DEF,KC,NaN,2018-12-24T04:26:43.400Z,40,40.61,33.44,0.02,0.00,0.40,266.10,161.25,2.814343,0.006429,-0.018939,-1.243547,39.0
1073936,2018122314,4239,10,None,2558865.0,Chris Carson,32.0,RB,home,OFF,SEA,NaN,2018-12-24T04:26:43.400Z,40,34.59,26.27,0.00,0.00,0.00,106.43,148.15,2.585705,0.000000,-0.000000,0.000000,39.0


In [8]:
motion = track_df[['dis', 's', 'v_x', 'v_y', 'a']]
motion.describe()

,dis,s,v_x,v_y,a
count,536076.000000,536076.000000,499504.000000,499504.000000,536076.000000
mean,0.305874,3.071834,1.363173,-0.129083,2.219274
std,0.234060,2.278834,2.661777,2.317083,1.585410
min,0.000000,0.000000,-8.209913,-10.105628,0.000000
25%,0.110000,1.140000,-0.087530,-1.088854,1.040000
50%,0.270000,2.790000,0.924624,0.000000,2.010000
75%,0.460000,4.690000,3.062776,0.845777,3.150000
max,5.540000,27.050000,9.697260,9.292045,25.470000


In [7]:
track_df.loc[track_df.position=="OLB"].displayName.unique()

array(['Derrick Morgan', 'Harold Landry', 'Ryan Kerrigan',
       'Preston Smith', 'Pernell McPhee', 'Ryan Anderson',
       'Terrell Suggs', 'Matt Judon', 'Jatavis Brown', "Za'Darius Smith",
       'Kyle Emanuel', 'Tyus Bowser', 'Olivier Vernon', 'Clay Matthews',
       'Kyler Fackrell', 'Nigel Bradham', 'Kamu Grugier-Hill',
       'Vontaze Burfict', 'Nick Vigil', 'Kareem Martin', 'Lorenzo Carter',
       'Telvin Smith', 'Myles Jack', 'Kyle Van Noy', 'Anthony Barr',
       'Eric Wilson', 'Vic Beasley', "De'Vondre Campbell",
       'Lorenzo Alexander', 'Whitney Mercilus', 'Neville Hewitt',
       'Jordan Jenkins', 'Devon Kennard', 'Eli Harold', 'Romeo Okwara',
       'Jamie Collins', 'Ben Gedeon', 'Thomas Davis', 'Lavonte David',
       'Devante Bond', 'Damien Wilson', 'Connor Barwin', 'Vincent Rey',
       'Albert McClellan', 'Leonard Floyd', 'Samson Ebukam', 'A.J. Klein',
       'Alex Anzalone', 'Bud Dupree', 'T.J. Watt', 'Craig Robertson',
       'Anthony Chickillo', 'Matt Longacre'

In [8]:
track_df.loc[track_df.position=="ILB"].displayName.unique()

array(['Wesley Woodyard', 'Jayon Brown', 'Mason Foster', 'Zach Brown',
       'Kamalei Correa', 'Daren Bates', 'C.J. Mosley', 'Patrick Onwuasor',
       'B.J. Goodson', 'Tae Davis', 'Antonio Morrison', 'Blake Martinez',
       'Ukeme Eligwe', 'Benardrick McKinney', 'Zach Cunningham',
       'Ahmad Thomas', 'Matthew Adams', 'Zaire Franklin',
       'Avery Williamson', 'Dylan Cole', 'Christian Jones',
       'Deone Bucannon', 'Haason Reddick', 'Danny Trevathan',
       'Mark Barron', 'Cory Littleton', 'Demario Davis', 'L.J. Fort',
       'Vince Williams', 'Jon Bostic', 'Bryce Hager', 'Ramik Wilson',
       'Reggie Ragland', 'Nicholas Morrow', 'Todd Davis',
       'Brandon Marshall'], dtype=object)

In [9]:
track_df.loc[track_df.position=="MLB"].displayName.unique()

array(['Hayes Pullard', 'Jaylon Smith', 'Jordan Hicks', 'Hardy Nickerson',
       'Deion Jones', "Dont'a Hightower", 'Elandon Roberts',
       'Jarrad Davis', 'Mike Hull', 'Joe Schobert', 'Luke Kuechly',
       'Adarius Taylor', 'Kentrell Brothers', 'Mark Nzeocha',
       'Bobby Wagner', 'Anthony Hitchens', 'Marquel Lee'], dtype=object)